In [36]:
import oracledb
import json

In [35]:
def creat_oracle_connection():
    dsn_str = oracledb.makedsn("oracle.fiap.com.br", 1521, "ORCL")
    user = "rm556426"
    password = "300103"
    con = oracledb.connect(user=user,password=password,dsn=dsn_str)
    return con

In [ ]:
def creat_oracle_connection_with_sql_alchemy():
    dsn_str = oracledb.makedsn("oracle.fiap.com.br", 1521, "ORCL")
    user = "rm556426"
    password = "300103"
    con = oracledb.connect(user=user,password=password,dsn=dsn_str)
    return con

In [37]:
con = creat_oracle_connection()
cur = con.cursor()

In [4]:
# result = cur.execute("SELECT name FROM sqlite_master")
# exists = any("movie" in record for record in result)

In [18]:
# if not exists:
#     cur.execute("CREATE TABLE movie(title, year, score)")
cur.execute("CREATE TABLE movie(title VARCHAR2(100), year NUMBER(4), score NUMBER(2,1))")
 

In [5]:
res = cur.execute("SELECT name FROM sqlite_master")
# res.fetchone()
res.fetchall()

[('movie',)]

In [19]:
cur.execute(
    """
INSERT INTO movie(title, year, score)
VALUES ('The Shawshank Redemption', 1994, 9.3)
            """
)

In [20]:
cur.execute("SELECT * from movie").fetchall()

[('The Shawshank Redemption', 1994, 9.3)]

In [21]:
con.commit()

In [22]:
result = cur.execute("SELECT * FROM movie")
result.fetchall()

[('The Shawshank Redemption', 1994, 9.3)]

In [27]:
data = [
        ('Monty Python and the Holy Grail', 1975, 8.2),
        ('And Now for Something Completely Different', 1971, 7.5)
]
cur.executemany("INSERT INTO movie VALUES(:1, :2, :3)", data)
con.commit()

In [28]:
result = cur.execute("SELECT * FROM movie")
result.fetchall()

[('The Shawshank Redemption', 1994, 9.3),
 ('Monty Python and the Holy Grail', 1975, 8.2),
 ('And Now for Something Completely Different', 1971, 7.5)]

In [29]:
result = cur.execute("SELECT * FROM movie WHERE score > 8.0")
result.fetchall()

[('The Shawshank Redemption', 1994, 9.3),
 ('Monty Python and the Holy Grail', 1975, 8.2)]

In [30]:
cur.execute("UPDATE movie SET score = 9.0 WHERE title = 'The Shawshank Redemption'")

In [31]:
con.commit()

In [39]:
cur.execute("SELECT * FROM movie WHERE title = 'The Shawshank Redemption'").fetchall()

InterfaceError: DPY-1001: not connected to database

In [38]:
con.close()

# Utilizando o pandas para trabalhar com o banco de dados

In [40]:
import pandas as pd

In [42]:
bd_carros = pd.read_csv("bd.csv")  # , parse_dates=["Report Received Date"])
bd_carros["Potentially Affected"] = (
    bd_carros["Potentially Affected"].fillna(0).astype("int")
)
bd_carros.head()

,Report Received Date,Manufacturer,Component,Recall Type,Potentially Affected
0,01/06/2023,Triple E Recreational Vehicles,ELECTRICAL SYSTEM,Vehicle,341
1,01/05/2023,"Volvo Car USA, LLC",STEERING,Vehicle,74
2,12/29/2022,"Volkswagen Group of America, Inc.",ELECTRICAL SYSTEM,Vehicle,1042
3,12/29/2022,Indian Motorcycle Company,STRUCTURE,Vehicle,4653
4,12/29/2022,Hendrickson USA. L.L.C.,"SERVICE BRAKES, AIR",Equipment,12


In [43]:
bd_carros.columns = [r.lower().replace(" ", "_") for r in bd_carros.columns]

In [44]:
bd_carros.head()

,report_received_date,manufacturer,component,recall_type,potentially_affected
0,01/06/2023,Triple E Recreational Vehicles,ELECTRICAL SYSTEM,Vehicle,341
1,01/05/2023,"Volvo Car USA, LLC",STEERING,Vehicle,74
2,12/29/2022,"Volkswagen Group of America, Inc.",ELECTRICAL SYSTEM,Vehicle,1042
3,12/29/2022,Indian Motorcycle Company,STRUCTURE,Vehicle,4653
4,12/29/2022,Hendrickson USA. L.L.C.,"SERVICE BRAKES, AIR",Equipment,12


In [45]:
bd_carros.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26590 entries, 0 to 26589
Data columns (total 5 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   report_received_date  26590 non-null  object
 1   manufacturer          26590 non-null  object
 2   component             26590 non-null  object
 3   recall_type           26590 non-null  object
 4   potentially_affected  26590 non-null  int64 
dtypes: int64(1), object(4)
memory usage: 1.0+ MB


In [46]:
con = creat_oracle_connection()
cur = con.cursor()

In [48]:
cur.execute(
    """
    CREATE TABLE cars(
        report_received_date VARCHAR2(10),
        manufacturer VARCHAR2(100),
        component VARCHAR2(100),
        recall_type VARCHAR2(100),
        potentially_affected NUMBER(10)
        )
    """
)


In [49]:
data = bd_carros.head().to_records(index=False).tolist()
cur.executemany(
    """
    INSERT INTO cars VALUES(:1,:2,:3,:4,:5)
    """,
    data,
)
con.commit()

In [50]:
bd_carros.report_received_date = bd_carros.report_received_date.apply(pd.to_datetime)

In [52]:
#bd_carros.to_sql("cars2", con, index=False)

In [53]:
bd_carros.shape

(26590, 5)

In [54]:
pd.read_sql(
    """
    SELECT *
    FROM cars2
    WHERE potentially_affected >= 20000
    """,
    con,
)

C:\Users\felip\AppData\Local\Temp\ipykernel_25320\1066276391.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql(


DatabaseError: Execution failed on sql '
    SELECT *
    FROM cars2
    WHERE potentially_affected >= 20000
    ': ORA-00942: table or view does not exist
Help: https://docs.oracle.com/error-help/db/ora-00942/